In [13]:
import operator
from typing import Annotated, TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage

# 1. Define the Shared State (Memory)
class AgentState(TypedDict):
    # 'Annotated' with 'operator.add' ensures messages are appended, not overwritten
    messages: Annotated[list[BaseMessage], operator.add]
    project_idea: str
    plan: str
    review_feedback: str

# 2. Initialize the Model
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# 3. Node 1: Planner Agent
def planner_node(state: AgentState):
    prompt = f"As a Lead Data Scientist, create a test plan for: {state['project_idea']}"
    response = llm.invoke([SystemMessage(content="Create a detailed technical plan."), HumanMessage(content=prompt)])
    return {"plan": response.content, "messages": [response]}

# 4. Node 2: Critic Agent (The "Reviewer")
def reviewer_node(state: AgentState):
    prompt = f"Review this plan for missing tools or unrealistic timelines: {state['plan']}"
    response = llm.invoke([SystemMessage(content="Be critical. Find 2 improvements."), HumanMessage(content=prompt)])
    return {"review_feedback": response.content, "messages": [response]}

# 5. Build the Graph
workflow = StateGraph(AgentState)

workflow.add_node("planner", planner_node)
workflow.add_node("reviewer", reviewer_node)

# Define the flow: Start -> Planner -> Reviewer -> End
workflow.add_edge(START, "planner")
workflow.add_edge("planner", "reviewer")
workflow.add_edge("reviewer", END)

# Compile the Agent
app = workflow.compile()
print("Advanced Agentic Workflow Compiled!")

Advanced Agentic Workflow Compiled!


In [14]:
from langgraph.checkpoint.memory import MemorySaver # Temporary memory for the session

# 1. Setup Memory (This allows the agent to 'wait' for you)
memory = MemorySaver()

# 2. Define a simple Approval Node
def human_approval_node(state: AgentState):
    # This node is a placeholder; the 'interrupt' happens before it
    print("--- HUMAN APPROVAL GRANTED ---")
    return state

# 3. Re-build the Graph with Interruption
workflow = StateGraph(AgentState)

workflow.add_node("planner", planner_node)
workflow.add_node("reviewer", reviewer_node)
workflow.add_node("human_approval", human_approval_node)

workflow.add_edge(START, "planner")
workflow.add_edge("planner", "reviewer")
workflow.add_edge("reviewer", "human_approval")
workflow.add_edge("human_approval", END)

# CRITICAL: We tell the graph to stop before 'human_approval'
app = workflow.compile(checkpointer=memory, interrupt_before=["human_approval"])

In [15]:
# Create a unique thread ID for this conversation
config = {"configurable": {"thread_id": "EPAM_Interview_Demo_1"}}

initial_input = {"project_idea": "Airlines Cargo Weight Optimization using Linear Regression", "messages": []}

# The agent will run until it hits the 'human_approval' node and then STOP.
for event in app.stream(initial_input, config):
    print(event)

{'planner': {'plan': 'As the Lead Data Scientist for the Airlines Cargo Weight Optimization project, this test plan outlines the strategy and specific tests required to ensure the robustness, accuracy, reliability, and business value of our Linear Regression model.\n\n---\n\n## Test Plan: Airlines Cargo Weight Optimization using Linear Regression\n\n**Document Version:** 1.0\n**Date:** October 26, 2023\n**Author:** [Your Name/Lead Data Scientist]\n\n---\n\n### 1. Introduction\n\nThis document details the test plan for the Airlines Cargo Weight Optimization model, which utilizes Linear Regression to predict optimal cargo weights for various flight segments, aircraft types, and operational conditions. The primary goal of this model is to maximize cargo revenue while adhering to critical safety and operational constraints (e.g., maximum takeoff weight, center of gravity limits, fuel efficiency).\n\nThe purpose of this test plan is to ensure that the developed model is:\n*   **Accurate:** 

In [17]:
# This acts as the 'Human' saying 'Yes, I approve'
app.invoke(None, config) 
print("Project Finalized!")

Project Finalized!
